# Trabajo Práctico 1: Enunciado


El trabajo práctico 1 de la materia se basa en el análisis de los tweets del set de datos de la competencia: https://www.kaggle.com/c/nlp-getting-started.  

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0)

El objetivo del primer TP es realizar un análisis exploratorio del set de datos. Queremos ver qué cosas podemos descubrir sobre los datos que puedan resultar interesantes. Estas cosas pueden estar relacionadas al objetivo del TP2 (predecir si un cierto tweet es real o no) o no, ambas son de interés.

Los requisitos de la primera entrega son los siguientes:

- El análisis debe estar hecho en Python Pandas o R.
- El análisis debe entregarse en formato pdf vía gradescope. En el informe no va código.
- Informar el link a un repositorio Github en donde pueda bajarse el código completo para generar el análisis.

La evaluación del TP se realizará en base al siguiente criterio:

- Originalidad del análisis exploratorio. 
- Calidad del reporte. ¿Está bien escrito? ¿Es claro y preciso? 
- Calidad del análisis exploratorio: qué tipo de preguntas se hacen y de qué forma se responden, ¿es la respuesta clara y concisa con respecto a la pregunta formulada? 
- Calidad de las visualizaciones presentadas.
  - ¿Tienen todos los ejes su rótulo?
  - ¿Tiene cada visualización un título?
  - ¿Es entendible la visualización sin tener que leer la explicación?
  - ¿El tipo de plot elegido es adecuado para lo que se quiere visualizar?
  - ¿Es una visualización interesante?
  - ¿El uso del color es adecuado?
  - ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
  - ¿La visualización es consistente con los datos?
- Conclusiones presentadas.


In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tweets = pd.read_csv('../data/train.csv', encoding='utf-8')
tweets.head()

In [ ]:
tweets.info()
# keyword tiene 61 nulls
# location tiene 2533 nulls

In [ ]:
# IMPORTANTE esto reemplazando todos los NaN por None
tweets = tweets.fillna('None')

In [ ]:
tweets.info()

### Análisis de los textos

In [ ]:
!pip install nltk
!pip install stopwords

In [ ]:
#import nltkfrom nltk.corpus 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize



#### Eliminando los links del texto y dejandolos en una columna a parte por si sirven a futuro
Quiero ver si hay relación entre cantidad de tweets fakes y no fakes y la cantidad de links (DESCARTADO)

In [ ]:
# primera normalización del texto, pasamos a lowercase 
tweets['normalized_text'] = tweets.text.str.lower()

Limpieza de links

In [ ]:
# nueva columna con el total de links

import re
URLPATTERN = r'(https?://\S+)' 

tweets['urls'] = tweets.normalized_text.apply(lambda x: re.findall(URLPATTERN, x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(URLPATTERN,"", x))
# cuento la cantidad de links en los tweets
tweets['url_count'] = tweets.urls.str.len()

tweets[(tweets['url_count'] > 1)].head()

#### Contador de hashtags 
Quiero ver si hay relación entre cantidad de tweets fakes y no fakes y la cantidad de hashtags 

In [ ]:
# cuento la cantidad de hashtags en los tweets
# nueva columna con el total de hashtags, y los hashtags
tweets['hashtags'] = tweets.normalized_text.apply(lambda x: re.findall(r"#(\w+)", x))
tweets['hashtags_count'] = tweets.hashtags.str.len()
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"#(\w+)","", x))
tweets[(tweets['hashtags_count'] > 1)].head()

#### Contador de tags 
Quiero ver si hay relación entre cantidad de tweets fakes y no fakes y la cantidad de tags 

In [ ]:
# cuento la cantidad de ags en los tweets
# nueva columna con el total de tags, y los tags
tweets['tags'] = tweets.text.str.lower().apply(lambda x: re.findall(r"@(\w+)", x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"@(\w+)","", x))
tweets['tags_count'] = tweets.tags.str.len()
tweets[(tweets['tags_count'] > 1)].head()


In [ ]:
import string
# replacing the punctuations with no space, 
# which in effect deletes the punctuation marks 
translator = str.maketrans('', '', string.punctuation)

def remove_punctuation(text):
    '''a function for removing punctuation'''    
    # return the text stripped of punctuation marks
    return text.translate(translator)

In [ ]:
def remove_htmlsymbols(text):
    soup = BeautifulSoup(unescape(text))
    return soup.text

In [ ]:
!pip install bs4

In [ ]:
#eliminamos cualquier signo de puntuacion y textos de tipo html como &amp
from bs4 import BeautifulSoup
from html import unescape

tweets['normalized_text'] = tweets.normalized_text.apply(remove_htmlsymbols)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_punctuation)


In [ ]:
# convertimos el texto en listado de palabras y despues borramos las stop words
tweets['words'] = tweets.normalized_text.str.split()
stop_words = stopwords.words('english')
tweets['normalized_words'] = tweets['words'].apply(lambda x: [item for item in x if item not in stop_words])
# eliminar las stop words del texto normalizado
tweets['normalized_text'] = [' '.join(map(str, l)) for l in tweets['normalized_words']]
tweets.head()

## Wordcloud con texto normalizado

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator

Wordcloud para textos fake

In [ ]:
# me quedo con todos los textos fakes
fakes= tweets[tweets.target == 0]
fakes_text = " ".join(review for review in fakes.normalized_text)

In [ ]:
# Create and generate a word cloud image:
wordcloud_fake = WordCloud(max_font_size=80, max_words=500, background_color="white").generate(fakes_text)

# Display the generated image:
plt.imshow(wordcloud_fake, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud_fake.to_file("wordcloud_fake.png")


Wordcloud para textos reales

In [ ]:
# me quedo con todos los textos reales
reales= tweets[tweets.target == 1]
reales_text = " ".join(review for review in reales.normalized_text)

In [ ]:
#?WordCloud

In [ ]:
# Create and generate a word cloud image:
wordcloud_real = WordCloud(max_font_size=80, max_words=500, background_color="white").generate(reales_text)

# Display the generated image:
plt.imshow(wordcloud_real, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud_real.to_file("wordcloud_real.png")


#### Bag of Words

In [ ]:
from collections import Counter
# contamos la cantidad de palabras de cada tweet
tweets['words_counter'] = tweets.normalized_words.apply(Counter)

In [ ]:
tweets.head()

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en cada row
tweets['word_max_appearance'] = tweets.words_counter.apply( lambda x: max(x) if x else None)  
tweets.head()

In [ ]:
tweets['word_max_appearance_count'] =  tweets.words_counter.apply( lambda x: max(x.values()) if x else 0)
tweets.head()

In [ ]:
tweets['word_max_appearance_count'].value_counts()

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en todo el dataset para los fake

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en todo el dataset para los posta

In [ ]:
tweets.head()

### Análisis de los keywords

In [ ]:
tweets['keywords_normalized'] = tweets.keyword.str.replace('%20',' ')

In [ ]:
tweets.keywords_normalized.value_counts()

In [ ]:
# Agrupo por keyword y cuento la cantidad de apariciones que tienen y el promedio del target para sacar una suerte de "probabilidad de catastrofe"
ag = tweets.groupby('keywords_normalized').agg({'text':'count', 'target':'mean'}).rename(columns={'text':'Apariciones', 'target':'Probabilidad de Catastrofe'})
ag.sort_values('Probabilidad de Catastrofe', ascending=False).head(20)


In [ ]:
probability = tweets.groupby('keywords_normalized').agg({'target':'mean'}).rename(columns={'target':'Probabilidad de Catastrofe'})
probability.sort_values('Probabilidad de Catastrofe', ascending=False).head(20)

In [ ]:
probability.sort_values('Probabilidad de Catastrofe', ascending=False).head(50).plot(kind='bar',figsize=(16,8),rot=85,title='Probabilidad de catástrofe según keyword')
ax=plt.gca()
ax.set_ylabel('Probabilidad');
ax.set_xlabel('Keyword')

### Matriz de correlación para ver cómo se relacionan las variables entre sí

In [ ]:
tweets.corr()